In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import dash_table
from dash.dependencies import Input, Output
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

from CRUD import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacUser"
password = "Rhspanthers58"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


html.Br(),
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={"height":"2""00px"}, )),        
    html.A(href="https://www.snhu.edu"),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H3("Created by Paul Gardner")),
    html.H6("Select a rescue type from the options below:"),
    html.Hr(),
    html.Div(
        className='row',
        style={'flex-wrap': 'nowrap'},
        children=[
            dcc.RadioItems(
            id='button', 
            options=[
            {"label": "All Dogs", "value": "Reset"},
            {"label": "Water Rescue", "value": "Water Rescue"},
            {"label": "Mountain Rescue", "value": "Mountain Rescue"},
            {"label": "Disaster Tracking", "value": "Disaster Rescue"},
        ],
        value='Reset',
        labelStyle={"display": "inline-block"},
        )
    ]),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action ="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        className='row',
         style={'display' : 'flex'},
             children=[
        html.Div( 
            id='graph-id',
            className='col s12 m6',
        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'} 
    for i in selected_columns]

#Buttons callback
@app.callback(
    Output('datatable-id',"data"),
     [Input('button', 'value')])
def on_click(value):
    if value == "Reset":
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog"}))
        return df.to_dict("records")
    elif value == "Water Rescue":
        df = pd.DataFrame(shelter.read({"animal_type":"Dog","breed":{"$in"
            :["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]}, 
            "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))
        return df.to_dict("records")                  
    elif value == "Mountain Rescue":
        df = pd.DataFrame(shelter.read({"animal_type":"Dog","breed":{"$in"
            :["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, 
            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26},
            "age_upon_outcome_in_weeks":{"$lte":156}}))
        return df.to_dict("records")
    elif value == "Disaster Rescue": 
        df = pd.DataFrame(shelter.read({"animal_type":"Dog","breed":{"$in"
            :["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]}, 
            "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20},
            "age_upon_outcome_in_weeks":{"$lte":300}}))
        return df.to_dict("records") 

#graph callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)     
    return [
        dcc.Graph(
            figure = px.pie(dff, names= 'breed')
        )
    ]
    
#Geo map callback
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map( style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app